In [1]:
!pip install google-api-python-client pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 4.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import math

# Define a function to calculate total quota usage for a given number of videos
def calculate_quota_usage(num_videos):
    # Quota for video search
    video_search_requests = math.ceil(num_videos / 50)
    video_search_quota = video_search_requests * 500
    
    # Quota for comments (1 unit per video to get top 100 comments)
    comments_quota = num_videos * 1
    
    # Total quota usage
    total_quota = video_search_quota + comments_quota
    
    return total_quota

# Now, find the maximum number of videos that keep total quota under 9000 units
target_quota = 9000
max_videos = 0

for videos in range(1, 5000):  # Try different values for videos
    total_quota = calculate_quota_usage(videos)
    if total_quota <= target_quota:
        max_videos = videos
    else:
        break

max_videos, calculate_quota_usage(max_videos)  # Return the number of videos and the total quota usage

(800, 8800)

In [1]:
import pandas as pd
import os
import time
import logging
from googleapiclient.discovery import build
import config  # Import the config file
from googleapiclient.errors import HttpError

# Set up your YouTube API key
api_key = config.YOUTUBE_API_KEY

# Build the YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

# Set up logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Ensure a directory exists to store the files
output_folder = 'youtube_data'
os.makedirs(output_folder, exist_ok=True)  # Create folder if it doesn't exist

# Function to get video details by a search term (with pagination, likes, and error handling)
def get_video_details(search_query, max_results=800):
    videos = []
    next_page_token = None

    logging.info(f"Starting to scrape video details for search term: {search_query}")

    while len(videos) < max_results:
        try:
            # Step 1: Search for videos using the search term
            request = youtube.search().list(
                q=f'{search_query} 2024 "presidential election" OR "US election" OR "US elections"',
                part='snippet',
                type='video',
                maxResults=min(50, max_results - len(videos)),  # Fetch up to 50 results per request
                pageToken=next_page_token,
                relevanceLanguage="en",  # Filter by English content
                regionCode="US",  # Prioritize US-based content
                order="relevance",  # Order results by relevance to the search term
                safeSearch="moderate",  # Optional: filter out inappropriate content
            )
            response = request.execute()

            video_ids = [item['id']['videoId'] for item in response['items']]  # Collect video IDs

            # Step 2: Fetch statistics (likes, views, etc.) for the collected video IDs
            stats_request = youtube.videos().list(
                part='statistics',
                id=','.join(video_ids)  # Pass video IDs as a comma-separated string
            )
            stats_response = stats_request.execute()

            # Step 3: Combine snippet and statistics data
            for item, stats in zip(response['items'], stats_response['items']):
                video = {
                    'video_id': item['id']['videoId'],
                    'title': item['snippet']['title'],
                    'description': item['snippet']['description'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_title': item['snippet']['channelTitle'],
                    'like_count': stats['statistics'].get('likeCount', 0),  # Likes
                    'view_count': stats['statistics'].get('viewCount', 0)   # Views (optional)
                }

                # Filter to include only videos explicitly mentioning the election
                if 'election' in video['title'].lower() or 'election' in video['description'].lower():
                    videos.append(video)

            logging.info(f"Scraped {len(videos)} videos so far...")

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more pages, stop the loop

        except Exception as e:
            logging.error(f"Error occurred while fetching videos: {e}")
            time.sleep(5)  # Sleep for 5 seconds and retry

    logging.info(f"Finished scraping video details. Total videos scraped: {len(videos)}")
    return pd.DataFrame(videos)

# Function to get top 500 comments from a video (with pagination and error handling)
def get_video_comments(video_id, max_results=500):
    comments = []
    next_page_token = None

    logging.info(f"Starting to scrape comments for video: {video_id}")

    while len(comments) < max_results:
        try:
            request = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=min(100, max_results - len(comments)),  # Fetch up to 100 comments per request
                pageToken=next_page_token
            )
            response = request.execute()

            for item in response['items']:
                comment = {
                    'author': item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                    'text': item['snippet']['topLevelComment']['snippet']['textDisplay'],
                    'published_at': item['snippet']['topLevelComment']['snippet']['publishedAt'],
                    'like_count': item['snippet']['topLevelComment']['snippet']['likeCount']
                }
                comments.append(comment)

            logging.info(f"Scraped {len(comments)} comments so far for video {video_id}...")

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # No more pages, stop the loop

        except HttpError as e:
            # Check if the error is due to comments being disabled
            if e.resp.status == 403 and 'commentsDisabled' in str(e):
                logging.warning(f"Encountered 403 Forbidden with reason 'commentsDisabled' for video {video_id}")
                break  # Do not retry for this video
            else:
                logging.error(f"Error occurred while fetching comments for video {video_id}: {e}")
                time.sleep(5)  # Sleep for 5 seconds and retry

    logging.info(f"Finished scraping comments for video {video_id}. Total comments: {len(comments)}")
    return pd.DataFrame(comments)

# Example: Scrape video details for a search term (up to 800 videos)
search_term = 'U.S. elections 2024'
try:
    videos_df = get_video_details(search_term, max_results=800)
    video_file_path = os.path.join(output_folder, 'youtube_videos.csv')
    videos_df.to_csv(video_file_path, index=False)
    logging.info(f"Video details saved to: {video_file_path}")
except Exception as e:
    logging.error(f"Error occurred while saving video details: {e}")

# Scrape top 500 comments for **each video** in the list of scraped videos
for idx, row in videos_df.iterrows():
    try:
        video_id = row['video_id']
        comments_df = get_video_comments(video_id, max_results=500)
        comments_file_path = os.path.join(output_folder, f'youtube_comments_{video_id}.csv')
        comments_df.to_csv(comments_file_path, index=False)
        logging.info(f"Comments for video {video_id} saved to: {comments_file_path}")
    except Exception as e:
        logging.error(f"Error occurred while saving comments for video {video_id}: {e}")

2024-10-16 12:19:46,771 - INFO - Starting to scrape video details for search term: U.S. elections 2024
2024-10-16 12:19:47,800 - INFO - Scraped 46 videos so far...
2024-10-16 12:19:48,634 - INFO - Scraped 88 videos so far...
2024-10-16 12:19:49,497 - INFO - Scraped 121 videos so far...
2024-10-16 12:19:50,149 - INFO - Scraped 160 videos so far...
2024-10-16 12:19:50,909 - INFO - Scraped 206 videos so far...
2024-10-16 12:19:51,918 - INFO - Scraped 246 videos so far...
2024-10-16 12:19:52,855 - INFO - Scraped 294 videos so far...
2024-10-16 12:19:53,777 - INFO - Scraped 338 videos so far...
2024-10-16 12:19:54,707 - INFO - Scraped 386 videos so far...
2024-10-16 12:19:55,737 - INFO - Scraped 433 videos so far...
2024-10-16 12:19:56,545 - INFO - Scraped 479 videos so far...
2024-10-16 12:19:57,455 - INFO - Scraped 504 videos so far...
2024-10-16 12:19:57,456 - INFO - Finished scraping video details. Total videos scraped: 504
2024-10-16 12:19:57,485 - INFO - Video details saved to: youtub

2024-10-16 12:20:03,661 - INFO - Starting to scrape comments for video: l7e1Xa0WLsI
2024-10-16 12:20:03,941 - INFO - Scraped 100 comments so far for video l7e1Xa0WLsI...
2024-10-16 12:20:04,130 - INFO - Scraped 128 comments so far for video l7e1Xa0WLsI...
2024-10-16 12:20:04,131 - INFO - Finished scraping comments for video l7e1Xa0WLsI. Total comments: 128
2024-10-16 12:20:04,137 - INFO - Comments for video l7e1Xa0WLsI saved to: youtube_data/youtube_comments_l7e1Xa0WLsI.csv
2024-10-16 12:20:04,138 - INFO - Starting to scrape comments for video: s_NeINDX1C8
2024-10-16 12:20:04,425 - INFO - Scraped 100 comments so far for video s_NeINDX1C8...
2024-10-16 12:20:04,752 - INFO - Scraped 200 comments so far for video s_NeINDX1C8...
2024-10-16 12:20:05,080 - INFO - Scraped 300 comments so far for video s_NeINDX1C8...
2024-10-16 12:20:05,315 - INFO - Scraped 400 comments so far for video s_NeINDX1C8...
2024-10-16 12:20:05,316 - INFO - Finished scraping comments for video s_NeINDX1C8. Total comm

2024-10-16 12:20:12,608 - INFO - Scraped 500 comments so far for video P3Ne1uI22No...
2024-10-16 12:20:12,609 - INFO - Finished scraping comments for video P3Ne1uI22No. Total comments: 500
2024-10-16 12:20:12,618 - INFO - Comments for video P3Ne1uI22No saved to: youtube_data/youtube_comments_P3Ne1uI22No.csv
2024-10-16 12:20:12,619 - INFO - Starting to scrape comments for video: 0N5sn9WfDlQ
2024-10-16 12:20:12,747 - WARNING - Encountered 403 Forbidden with reason "commentsDisabled"
2024-10-16 12:20:12,748 - WARNING - Encountered 403 Forbidden with reason 'commentsDisabled' for video 0N5sn9WfDlQ
2024-10-16 12:20:12,748 - INFO - Finished scraping comments for video 0N5sn9WfDlQ. Total comments: 0
2024-10-16 12:20:12,752 - INFO - Comments for video 0N5sn9WfDlQ saved to: youtube_data/youtube_comments_0N5sn9WfDlQ.csv
2024-10-16 12:20:12,753 - INFO - Starting to scrape comments for video: n4aRCA1qxnk
2024-10-16 12:20:13,013 - INFO - Scraped 100 comments so far for video n4aRCA1qxnk...
2024-10-

2024-10-16 12:20:21,052 - INFO - Comments for video CWMv8s_PfUI saved to: youtube_data/youtube_comments_CWMv8s_PfUI.csv
2024-10-16 12:20:21,053 - INFO - Starting to scrape comments for video: v3tVG9XMQ5c
2024-10-16 12:20:21,345 - INFO - Scraped 100 comments so far for video v3tVG9XMQ5c...
2024-10-16 12:20:21,592 - INFO - Scraped 200 comments so far for video v3tVG9XMQ5c...
2024-10-16 12:20:21,849 - INFO - Scraped 300 comments so far for video v3tVG9XMQ5c...
2024-10-16 12:20:22,151 - INFO - Scraped 400 comments so far for video v3tVG9XMQ5c...
2024-10-16 12:20:22,507 - INFO - Scraped 500 comments so far for video v3tVG9XMQ5c...
2024-10-16 12:20:22,508 - INFO - Finished scraping comments for video v3tVG9XMQ5c. Total comments: 500
2024-10-16 12:20:22,516 - INFO - Comments for video v3tVG9XMQ5c saved to: youtube_data/youtube_comments_v3tVG9XMQ5c.csv
2024-10-16 12:20:22,518 - INFO - Starting to scrape comments for video: 6xSAFkleDS0
2024-10-16 12:20:22,777 - INFO - Scraped 67 comments so far

2024-10-16 12:20:29,393 - INFO - Starting to scrape comments for video: RX2RIXqpZbA
2024-10-16 12:20:29,661 - INFO - Scraped 100 comments so far for video RX2RIXqpZbA...
2024-10-16 12:20:29,965 - INFO - Scraped 200 comments so far for video RX2RIXqpZbA...
2024-10-16 12:20:30,216 - INFO - Scraped 296 comments so far for video RX2RIXqpZbA...
2024-10-16 12:20:30,217 - INFO - Finished scraping comments for video RX2RIXqpZbA. Total comments: 296
2024-10-16 12:20:30,224 - INFO - Comments for video RX2RIXqpZbA saved to: youtube_data/youtube_comments_RX2RIXqpZbA.csv
2024-10-16 12:20:30,225 - INFO - Starting to scrape comments for video: RTRDclzdz2I
2024-10-16 12:20:30,431 - INFO - Scraped 18 comments so far for video RTRDclzdz2I...
2024-10-16 12:20:30,432 - INFO - Finished scraping comments for video RTRDclzdz2I. Total comments: 18
2024-10-16 12:20:30,437 - INFO - Comments for video RTRDclzdz2I saved to: youtube_data/youtube_comments_RTRDclzdz2I.csv
2024-10-16 12:20:30,439 - INFO - Starting to

...

2024-10-16 12:23:02,149 - INFO - Starting to scrape comments for video: gtv6HCTYubU
2024-10-16 12:23:02,298 - INFO - Scraped 0 comments so far for video gtv6HCTYubU...
2024-10-16 12:23:02,299 - INFO - Finished scraping comments for video gtv6HCTYubU. Total comments: 0
2024-10-16 12:23:02,303 - INFO - Comments for video gtv6HCTYubU saved to: youtube_data/youtube_comments_gtv6HCTYubU.csv
2024-10-16 12:23:02,304 - INFO - Starting to scrape comments for video: VLUvSZvNoIo
2024-10-16 12:23:02,434 - INFO - Scraped 0 comments so far for video VLUvSZvNoIo...
2024-10-16 12:23:02,435 - INFO - Finished scraping comments for video VLUvSZvNoIo. Total comments: 0
2024-10-16 12:23:02,438 - INFO - Comments for video VLUvSZvNoIo saved to: youtube_data/youtube_comments_VLUvSZvNoIo.csv
2024-10-16 12:23:02,439 - INFO - Starting to scrape comments for video: oEsMQUsS79g
2024-10-16 12:23:02,617 - INFO - Scraped 100 comments so far for video oEsMQUsS79g...
2024-10-16 12:23:02,805 - INFO - Scraped 200 comment

2024-10-16 12:23:10,323 - INFO - Comments for video 3-7yhFi6NNM saved to: youtube_data/youtube_comments_3-7yhFi6NNM.csv
2024-10-16 12:23:10,324 - INFO - Starting to scrape comments for video: JqPpLdv7GCc
2024-10-16 12:23:10,552 - INFO - Scraped 98 comments so far for video JqPpLdv7GCc...
2024-10-16 12:23:10,553 - INFO - Finished scraping comments for video JqPpLdv7GCc. Total comments: 98
2024-10-16 12:23:10,557 - INFO - Comments for video JqPpLdv7GCc saved to: youtube_data/youtube_comments_JqPpLdv7GCc.csv
2024-10-16 12:23:10,557 - INFO - Starting to scrape comments for video: 4zekY7ls-nE
2024-10-16 12:23:10,716 - INFO - Scraped 2 comments so far for video 4zekY7ls-nE...
2024-10-16 12:23:10,717 - INFO - Finished scraping comments for video 4zekY7ls-nE. Total comments: 2
2024-10-16 12:23:10,724 - INFO - Comments for video 4zekY7ls-nE saved to: youtube_data/youtube_comments_4zekY7ls-nE.csv
2024-10-16 12:23:10,726 - INFO - Starting to scrape comments for video: 1wjKU3Ann1w
2024-10-16 12:23

2024-10-16 12:23:19,009 - INFO - Comments for video m20sd_RlkfI saved to: youtube_data/youtube_comments_m20sd_RlkfI.csv
2024-10-16 12:23:19,010 - INFO - Starting to scrape comments for video: JBBzj956ahA
2024-10-16 12:23:19,156 - INFO - Scraped 0 comments so far for video JBBzj956ahA...
2024-10-16 12:23:19,157 - INFO - Finished scraping comments for video JBBzj956ahA. Total comments: 0
2024-10-16 12:23:19,162 - INFO - Comments for video JBBzj956ahA saved to: youtube_data/youtube_comments_JBBzj956ahA.csv
2024-10-16 12:23:19,166 - INFO - Starting to scrape comments for video: kW1HDx1cD_Y
2024-10-16 12:23:19,468 - INFO - Scraped 100 comments so far for video kW1HDx1cD_Y...
2024-10-16 12:23:19,813 - INFO - Scraped 200 comments so far for video kW1HDx1cD_Y...
2024-10-16 12:23:20,094 - INFO - Scraped 300 comments so far for video kW1HDx1cD_Y...
2024-10-16 12:23:20,384 - INFO - Scraped 400 comments so far for video kW1HDx1cD_Y...
2024-10-16 12:23:20,627 - INFO - Scraped 500 comments so far fo

2024-10-16 12:23:28,928 - INFO - Comments for video Hr551AGISXQ saved to: youtube_data/youtube_comments_Hr551AGISXQ.csv
2024-10-16 12:23:28,929 - INFO - Starting to scrape comments for video: 1ePtmM4X9UU
2024-10-16 12:23:29,290 - INFO - Scraped 2 comments so far for video 1ePtmM4X9UU...
2024-10-16 12:23:29,291 - INFO - Finished scraping comments for video 1ePtmM4X9UU. Total comments: 2
2024-10-16 12:23:29,297 - INFO - Comments for video 1ePtmM4X9UU saved to: youtube_data/youtube_comments_1ePtmM4X9UU.csv
2024-10-16 12:23:29,298 - INFO - Starting to scrape comments for video: DSfCp_A0qbM
2024-10-16 12:23:29,488 - INFO - Scraped 0 comments so far for video DSfCp_A0qbM...
2024-10-16 12:23:29,488 - INFO - Finished scraping comments for video DSfCp_A0qbM. Total comments: 0
2024-10-16 12:23:29,493 - INFO - Comments for video DSfCp_A0qbM saved to: youtube_data/youtube_comments_DSfCp_A0qbM.csv
2024-10-16 12:23:29,496 - INFO - Starting to scrape comments for video: K5LVekTMU40
2024-10-16 12:23:2

2024-10-16 12:23:35,977 - INFO - Finished scraping comments for video KdhNbqJIXko. Total comments: 1
2024-10-16 12:23:35,983 - INFO - Comments for video KdhNbqJIXko saved to: youtube_data/youtube_comments_KdhNbqJIXko.csv
2024-10-16 12:23:35,987 - INFO - Starting to scrape comments for video: N4pErHCWHrU
2024-10-16 12:23:36,175 - INFO - Scraped 1 comments so far for video N4pErHCWHrU...
2024-10-16 12:23:36,175 - INFO - Finished scraping comments for video N4pErHCWHrU. Total comments: 1
2024-10-16 12:23:36,179 - INFO - Comments for video N4pErHCWHrU saved to: youtube_data/youtube_comments_N4pErHCWHrU.csv
2024-10-16 12:23:36,180 - INFO - Starting to scrape comments for video: AmsL7Yv9VDA
2024-10-16 12:23:36,394 - INFO - Scraped 4 comments so far for video AmsL7Yv9VDA...
2024-10-16 12:23:36,395 - INFO - Finished scraping comments for video AmsL7Yv9VDA. Total comments: 4
2024-10-16 12:23:36,401 - INFO - Comments for video AmsL7Yv9VDA saved to: youtube_data/youtube_comments_AmsL7Yv9VDA.csv
2

In [2]:
videos_df

,video_id,title,description,published_at,channel_title,like_count,view_count
0,QHjptGT8mGc,Trump Live | Trump Attacks Iran | Trump Speech...,Donald Trump has been briefed by US intelligen...,2024-10-14T13:27:26Z,CNN-News18,12005,1313616
1,PIErb-yNEPw,How Does Race Impact Voting In the US? | US El...,How Does Race Impact Voting in The US? | US El...,2024-10-15T11:45:06Z,Firstpost,97,5807
2,qK2Zn5oCCq8,LIVE | Kamala Harris Slams Donald Trump In Pen...,US Democratic presidential candidate Kamala Ha...,2024-10-15T08:40:06Z,CNN-News18,270,22299
3,_6Px0WK2z-8,US Presidential Election 2024: Trump And Harri...,With just weeks to go until the US presidentia...,2024-10-15T16:15:22Z,WION,20,2056
4,z1XLk5Ba0qc,"US Election Final Stretch | October 14, 2024","Today on CNN 10, we check in on the Presidenti...",2024-10-13T23:00:36Z,CNN 10,1108,72468
...,...,...,...,...,...,...,...
499,v-MjLB0j6I4,US Presidential Elections 2024 | Donald Trump ...,US Presidential Elections 2024 | Donald Trump ...,2024-10-14T07:06:35Z,Wild Wolf Focus,11,881
500,cznL69iqOrI,Swing States Polls Trump vs Harris #trump #har...,Swing States Polls Trump vs Harris #trump #har...,2024-10-14T17:00:27Z,Orbital Order,7,2453
501,aJDROy58b9Q,Swing States Polls Trump vs Harris #trump #har...,Swing States Polls Trump vs Harris #trump #har...,2024-10-15T02:00:00Z,Orbital Order,6,378
502,Dxwsezra9I4,Kamala and Us US Elections 2024,A sub-economic view of America and Kamala Harr...,2024-10-15T23:57:51Z,Beri Lantum,0,0


In [3]:
comments_df

,author,text,published_at,like_count
0,@dennysbastos-9026,Mundo 🌍vei sem porteira 🌍🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🇧🇷🌍✍🏻...,2024-10-13T07:59:25Z,0
1,@chadgaming8288,"If, or rather, WHEN Donald Trump wins the Elec...",2024-10-12T17:47:42Z,0
2,@KAZVorpal,You can&#39;t trust anyone who says &quot;Cand...,2024-10-12T15:47:45Z,1
3,@KJSvitko,Trump is a con man fleecing the gullible and m...,2024-10-12T11:50:42Z,0
4,@DrMoto182,It takes a special kind of ignorance to want a...,2024-10-12T07:06:35Z,0
...,...,...,...,...
495,@anne-mariespencer6763,Vote TRUMP! Vote for Senators too!,2024-10-09T18:08:06Z,1
496,@Paproud62,I miss the peace &amp; prosperity we had when ...,2024-10-09T18:06:45Z,2
497,@joshferguson3113,Kick her a** Trump we all are Voting for you P...,2024-10-09T18:06:20Z,0
498,@adelethibault7411,Go Trump Go! You need in &amp; save the country!!,2024-10-09T18:05:42Z,1
